In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import os
import seaborn as sns
import random
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.impute import SimpleImputer

In [ ]:
data = pd.read_csv(r'C:\Users\kvsre\Downloadsspam_sms.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
#Check the data
data.info()

In [ ]:
#Check the missing values in the data
data.isnull().sum()

In [ ]:
data['label'].value_counts()

In [ ]:
# Prepare Data
df = data.groupby('label').size().reset_index(name='counts')
n = df['label'].unique().__len__()+1
all_colors = list(plt.cm.colors.cnames.keys())
random.seed(2000)
c = random.choices(all_colors, k=n)

# Plot Bars
plt.figure(figsize=(6,6), dpi= 80)
plt.bar(df['label'], df['counts'], color=c, width=.5)
for i, val in enumerate(df['counts'].values):
    plt.text(i, val, float(val), horizontalalignment='center', verticalalignment='bottom', fontdict={'fontweight':500, 'size':12})

# Decoration
plt.gca().set_xticklabels(df['label'],rotation=60,horizontalalignment='right')
plt.ylim(0, 4500)
plt.show()

In [ ]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = data[data['label']== 'ham']
df_minority = data[data['label']== 'spam']

# Downsample majority class and upsample the minority class
df_minority_upsampled = resample(df_minority, replace=True,n_samples=4000,random_state=123)
df_majority_downsampled = resample(df_majority, replace=False,n_samples=4000,random_state=123)

# Combine minority class with downsampled majority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority_downsampled])

# Display new class counts
df_upsampled['label'].value_counts()

In [ ]:
# shuffle the DataFrame rows
data= df_upsampled.sample(frac = 1)

In [ ]:

# Prepare Data
df = data.groupby('label').size().reset_index(name='counts')
n = df['label'].unique().__len__()+1
all_colors = list(plt.cm.colors.cnames.keys())
random.seed(2000)
c = random.choices(all_colors, k=n)

# Plot Bars
plt.figure(figsize=(6,6), dpi= 80)
plt.bar(df['label'], df['counts'], color=c, width=.5)
for i, val in enumerate(df['counts'].values):
    plt.text(i, val, float(val), horizontalalignment='center', verticalalignment='bottom', fontdict={'fontweight':500, 'size':12})

# Decoration
plt.gca().set_xticklabels(df['label'], rotation=60, horizontalalignment= 'right')

plt.ylim(0, 4500)
plt.show()

In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
data['text'].head(5)

In [ ]:
print("printing some random reviews")
print(9, data['text'].values[9])
print(34, data['text'].values[34])
print(147, data['text'].values[147])

In [ ]:
# Combining all the above stundents
from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [ ]:
preprocessed_text = preprocess_text(data['text'].values)

In [ ]:
print("printing some random reviews")
print(9, preprocessed_text[9])
print(34, preprocessed_text[34])
print(147, preprocessed_text[147])

In [ ]:
data["label"] = [1 if i == "spam" else 0 for i in data["label"]]
data.head(10)

In [ ]:
!pip install wordcloud

In [ ]:
#spam word cloud
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud

plt.figure(figsize=(10,10))
wc=WordCloud(max_words=2000,width=1600,height=800).generate(' '.join(data[data.label==1].text))
plt.imshow(wc , interpolation = 'bilinear')
plt.title("Spam Word Cloud")

In [ ]:
#ham word cloud
plt.figure(figsize=(10,10))
wc=WordCloud(max_words=2000,width=1600,height=800).generate(' '.join(data[data.label==0].text))
plt.imshow(wc , interpolation = 'bilinear')
plt.title("ham Word Cloud")

In [ ]:
import numpy as np
y=np.array(data['label'])

In [ ]:
from sklearn.model_selection import train_test_split
#Breaking into Train and test
X_train, X_test, y_train, y_test = train_test_split(preprocessed_text, y, test_size=0.3,stratify=y ,random_state=42)


In [ ]:
import pickle

In [ ]:
from sklearn import preprocessing

count_vect = CountVectorizer()
count_trans = count_vect.fit(X_train) # fit has to happen only on train data

# Dump the file
pickle.dump(count_trans, open(r"C:\Users\HP\Music\SMS_SPAM\FRONTEND\count_vect.pkl", "wb"))


# Testing phase
count = pickle.load(open(r"C:\Users\HP\Music\SMS_SPAM\FRONTEND\count_vect.pkl", 'rb'))

# we use the fitted CountVectorizer to convert the text to vector
X_train_count =count.transform(X_train)
X_test_count = count.transform(X_test)

#Normalize Data
X_train_count = preprocessing.normalize(X_train_count)
print("Train Data Size: ",X_train_count.shape)

#Normalize Data
X_test_count = preprocessing.normalize(X_test_count)
print("Test Data Size: ",X_test_count.shape)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

# Initialize and train CatBoost model
catboost_model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=0)
catboost_model.fit(X_train_count, y_train)

# Save the trained model
filename = r'C:\Users\HP\Music\SMS_SPAM\FRONTEND\catboost.pkl'
pickle.dump(catboost_model, open(filename, 'wb'))

# Predict on test and train data
pred_test = catboost_model.predict(X_test_count)
test_accuracy = accuracy_score(y_test, pred_test)
pred_train = catboost_model.predict(X_train_count)
train_accuracy = accuracy_score(y_train, pred_train)

print("Accuracy on Test data is " + str(test_accuracy))
print("Accuracy on Train data is " + str(train_accuracy))
print("---------------------------")

# Code for drawing seaborn heatmaps
class_names = ['HAM', 'SPAM']
df_heatmap = pd.DataFrame(confusion_matrix(y_test, pred_test), index=class_names, columns=class_names)
fig = plt.figure()
heatmap = sns.heatmap(df_heatmap, annot=True, fmt="d")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
class_names = ['HAM','SPAM']
print("Classification Report:")
print(classification_report(y_test, pred_test, target_names=class_names))

In [ ]:
original =  ["SPAM" if x==1 else "HAM" for x in y_test[:20]]
predicted = catboost_model.predict(X_test_count[:20])
pred = []

for i in predicted:
  if i == 1:
    k = "SPAM"
    pred.append(k)
  else:
    k = "HAM"
    pred.append(k)
# Creating a data frame
df = pd.DataFrame(list(zip(X_test[:20],original, pred,)),
               columns =['Text','original_Classlabel', 'predicted_classlebel'])
df

In [ ]:
from xgboost import XGBClassifier

# Initialize and train XGBoost model
xgb_model = XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.1, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_count, y_train)

# Save the trained model
filename = r'C:\Users\HP\Music\SMS_SPAM\FRONTEND\xgb.pkl'
pickle.dump(xgb_model, open(filename, 'wb'))

# Predict on test and train data
pred_test = xgb_model.predict(X_test_count)
test_accuracy = accuracy_score(y_test, pred_test)
pred_train = xgb_model.predict(X_train_count)
train_accuracy = accuracy_score(y_train, pred_train)

print("Accuracy on Test data is " + str(test_accuracy))
print("Accuracy on Train data is " + str(train_accuracy))
print("---------------------------")

# Code for drawing seaborn heatmaps
class_names = ['HAM', 'SPAM']
df_heatmap = pd.DataFrame(confusion_matrix(y_test, pred_test), index=class_names, columns=class_names)
fig = plt.figure()
heatmap = sns.heatmap(df_heatmap, annot=True, fmt="d")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
class_names = ['HAM','SPAM']
print("Classification Report:")
print(classification_report(y_test, pred_test, target_names=class_names))

In [ ]:
original =  ["SPAM" if x==1 else "HAM" for x in y_test[:20]]
predicted = xgb_model.predict(X_test_count[:20])
pred = []

for i in predicted:
  if i == 1:
    k = "SPAM"
    pred.append(k)
  else:
    k = "HAM"
    pred.append(k)
# Creating a data frame
df = pd.DataFrame(list(zip(X_test[:20],original, pred,)),
               columns =['Text','original_Classlabel', 'predicted_classlebel'])
df

In [ ]:
from lightgbm import LGBMClassifier

# Initialize and train LightGBM model
lgbm_model = LGBMClassifier(n_estimators=500, max_depth=6, learning_rate=0.1, objective='binary')
lgbm_model.fit(X_train_count, y_train)

# Save the trained model
filename = r'C:\Users\HP\Music\SMS_SPAM\FRONTEND\lgbm.pkl'
pickle.dump(lgbm_model, open(filename, 'wb'))

# Predict on test and train data
pred_test = lgbm_model.predict(X_test_count)
test_accuracy = accuracy_score(y_test, pred_test)
pred_train = lgbm_model.predict(X_train_count)
train_accuracy = accuracy_score(y_train, pred_train)

print("Accuracy on Test data is " + str(test_accuracy))
print("Accuracy on Train data is " + str(train_accuracy))
print("---------------------------")

# Code for drawing seaborn heatmaps
class_names = ['HAM', 'SPAM']
df_heatmap = pd.DataFrame(confusion_matrix(y_test, pred_test), index=class_names, columns=class_names)
fig = plt.figure()
heatmap = sns.heatmap(df_heatmap, annot=True, fmt="d")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
class_names = ['HAM','SPAM']
print("Classification Report:")
print(classification_report(y_test, pred_test, target_names=class_names))

In [ ]:
original =  ["SPAM" if x==1 else "HAM" for x in y_test[:20]]
predicted = lgbm_model.predict(X_test_count[:20])
pred = []

for i in predicted:
  if i == 1:
    k = "SPAM"
    pred.append(k)
  else:
    k = "HAM"
    pred.append(k)
# Creating a data frame
df = pd.DataFrame(list(zip(X_test[:20],original, pred,)),
               columns =['Text','original_Classlabel', 'predicted_classlebel'])
df

In [ ]:
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

# Initialize CatBoost Model
catboost_model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=0)
catboost_model.fit(X_train_count, y_train)

# Initialize LightGBM Model
lgbm_model = LGBMClassifier(n_estimators=500, max_depth=6, learning_rate=0.1, objective='binary')
lgbm_model.fit(X_train_count, y_train)

# Create a Voting Classifier (Soft Voting)
ensemble_model = VotingClassifier(estimators=[
    ('catboost', catboost_model),
    ('lgbm', lgbm_model)
], voting='soft')  # Soft Voting uses probability averaging

# Train the Ensemble Model
ensemble_model.fit(X_train_count, y_train)

# Predict on Test Data
pred_test = ensemble_model.predict(X_test_count)
test_accuracy = accuracy_score(y_test, pred_test)

# Predict on Train Data
pred_train = ensemble_model.predict(X_train_count)
train_accuracy = accuracy_score(y_train, pred_train)

print("Ensemble Model Accuracy on Test data: ", test_accuracy)
print("Ensemble Model Accuracy on Train data: ", train_accuracy)


In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier

# Initialize Models
catboost_model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=0)
xgb_model = XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.1, objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

# Train Models
catboost_model.fit(X_train_count, y_train)
xgb_model.fit(X_train_count, y_train)

# Voting Classifier (Soft Voting)
ensemble_model = VotingClassifier(estimators=[
    ('catboost', catboost_model),
    ('lgbm', lgbm_model)
], voting='soft', weights=[2, 1])

ensemble_model.fit(X_train_count, y_train)

# Save the trained model
filename = r'C:\Users\HP\Music\SMS_SPAM\FRONTEND\ensemble.pkl'
pickle.dump(ensemble_model, open(filename, 'wb'))

# Predict on test and train data
pred_test = ensemble_model.predict(X_test_count)
test_accuracy = accuracy_score(y_test, pred_test)
pred_train = ensemble_model.predict(X_train_count)
train_accuracy = accuracy_score(y_train, pred_train)

print("Accuracy on ensemble_model Test data is " + str(test_accuracy))
print("Accuracy on ensemble_model Train data is " + str(train_accuracy))
print("---------------------------")

# Code for drawing seaborn heatmaps
class_names = ['HAM', 'SPAM']
df_heatmap = pd.DataFrame(confusion_matrix(y_test, pred_test), index=class_names, columns=class_names)
fig = plt.figure()
heatmap = sns.heatmap(df_heatmap, annot=True, fmt="d")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
class_names = ['HAM','SPAM']
print("Classification Report:")
print(classification_report(y_test, pred_test, target_names=class_names))

In [ ]:
original =  ["SPAM" if x==1 else "HAM" for x in y_test[:20]]
predicted = ensemble_model.predict(X_test_count[:20])
pred = []

for i in predicted:
  if i == 1:
    k = "SPAM"
    pred.append(k)
  else:
    k = "HAM"
    pred.append(k)
# Creating a data frame
df = pd.DataFrame(list(zip(X_test[:20],original, pred,)),
               columns =['Text','original_Classlabel', 'predicted_classlebel'])
df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train)  # X_train should be your training text data
X_test_count = vectorizer.transform(X_test)        # X_test for testing


In [ ]:
def checkSpam(input_text): 
    # Display input
    print('Input:', input_text)

    # Vectorize input text
    input_vector = vectorizer.transform([input_text]).toarray()

    # Use the ensemble model for prediction
    prediction = ensemble_model.predict(input_vector)[0]

    # Convert numeric prediction to label
    label = "SPAM" if prediction == 1 else "HAM"

    # Display result
    print('Predicted Label:', label)


In [ ]:
checkSpam('Update_Now - Xmas Offer! Latest Motorola, SonyEricsson & Nokia & FREE Bluetooth! Double Mins & 1000 Txt on Orange. Call MobileUpd8 on 08000839402 or call2optout')

In [ ]:
checkSpam('hi today is my birthday, if u come to party u can won one lakhs of prize money?')

In [ ]:
checkSpam('Congratulations! Youve won a $1000 Walmart gift card. Go to [link] to claim now!')

In [ ]:
checkSpam('URGENT: Your account has been suspended. Verify at [phishy link] to restore access.')